#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Blog about Big Idea 2.4a and Hacks
- toc: true
- categories: [jupyter, Week 26]

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    2. SQLAlchemy db object


In [1]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

Comment on these items in the class, purpose and defintion.
- class User
    - The class that manages the actions in the 'users' table
- db.Model inheritance
    - It is a parameter of class User
    - Object oriented programming
- _init_ method
    - Is the constructor for the class User
    - Initializes the instance variables within object
- ```@property```, ```@<column>.setter```
    - ```@property``` is a getter method that extracts from the object
    - ```@<column>.setter``` allows the variable that was created to be updated
- create, read, update, delete methods
    - Allows the database and specific data inside to be created, read, updated, and deleted

In [2]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError

''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class NFL(db.Model):
    __tablename__ = 'NFL'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _position = db.Column(db.String(255), unique=False, nullable=False)
    _team = db.Column(db.String(255), unique=False, nullable=False)
    _number = db.Column(db.Integer, primary_key=False)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, position, team, number):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self._position = position
        self._team = team
        self._number = number

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    # a getter method, extracts mascot from object
    @property
    def position(self):
        return self._position
    
    # a setter function, allows mascot to be updated after initial object creation
    @position.setter
    def mascot(self, position):
        self._position = position

    # a getter method, extracts car from object
    @property
    def team(self):
        return self._team

    # a setter function, allows car to be updated after initial object creation
    @team.setter
    def team(self, team):
        self._team = team
    
    # a getter method, extracts mile from object
    @property
    def number(self):
        return self._number
    
    # a setter function, allows car to be updated after initial object creation
    @number.setter
    def number(self, number):
        self._number = number
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "position": self.position,
            "team": self.team,
            "number": self.number,
        }

    # CRUD update: updates user name, car, phone
    # returns self
    def update(self, name="", uid="", position="", team=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(position) > 0:
            self.position = position
        if len(team) > 0:
            self._team = team
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [5]:
def initNFL():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = NFL(name='Brandon Aiyuk', uid="baiyuk", position='Wide Receiver', number=11, team='San Francisco 49ers')
        u2 = NFL(name='Sauce Gardner', uid="sgardner", position='Cornerback', number=1, team='New York Jets')
        u3 = NFL(name='Justin Herbert', uid="jherbert", position='Quarterback', number=10, team='LA Chargers')
        u4 = NFL(name='Mark Andrews', uid="mandrews", position='Tight End', number=89, team='Baltimore Ravens')
        u5 = NFL(name='Jonathan Taylor', uid="jtaylor", position='Running Back', number=28, team='Indianapolis Colts')
        u6 = NFL(name='Daron Payne', uid="dpayne", position='Defensive Tackle', number=94, team='Washington Commanders')

        nfl_players = [u1, u2, u3, u4, u5, u6] # Changed variable name to nfl_players

        """Builds sample Track/note(s) data"""
        for player in nfl_players: # Changed variable name to player
            try:
                '''add Track to table'''
                object = player.create() # Changed variable name to player
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {player.uid}, or error.")

initNFL()

Created new uid baiyuk
Created new uid sgardner
Created new uid jherbert
Created new uid mandrews
Created new uid jtaylor
Created new uid dpayne


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
        - This receives a uid and looks to see if that uid exists in the database.
    2. user.password
        - If the uid is found, the passwords are checks and verified to make sure the password exists and is correct.

In [7]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = NFL.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    return True
        
check_credentials("sgardner")

True

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
        - A uid is inputted and it is checked whether the uid exists or not
        - If the user is found, then nothing is created
        - If there is no user with that uid, a new user is created
    2. user = User(...)
        - This only runs if the user is not found
        - A new user is created using the class User with a name, uid, and password
    3. user.dob and try/except
        - This only runs if the user is not found
        - If the dob that is inputted is the same as the date today, then an error appears
    4. user.create() and try/except
        - This only runs if the user is not found
        - If the object is not created, an error is raised; otherwise, the user is created

In [8]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your player's name:")
    position = input("Enter your player's position:")
    team = input("Enter your player's team:")
    number = input("Enter your player's jersey number:")
    
    # Initialize User object before date
    user = NFL(name=name, 
                uid=uid, 
                position=position,
                team=team,
                number=number
                )
    
    # create user.dob, fail with today as dob
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 7, 'name': 'Damar Hamlin', 'uid': 'dhamlin', 'position': 'Free Safety', 'team': 'Buffalo Bills', 'number': 3}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
        - This gets all the users from the database
    2. json_ready assignment, google List Comprehension
        - This reads all the users from the table and places them in a list using list comprehension
        - Each are turned into JSON data

In [14]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = NFL.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Brandon Aiyuk',
  'uid': 'baiyuk',
  'position': 'Wide Receiver',
  'team': 'San Francisco 49ers',
  'number': 11},
 {'id': 2,
  'name': 'Sauce Gardner',
  'uid': 'sgardner',
  'position': 'Cornerback',
  'team': 'New York Jets',
  'number': 1},
 {'id': 3,
  'name': 'Justin Herbert',
  'uid': 'jherbert',
  'position': 'Quarterback',
  'team': 'LA Chargers',
  'number': 10},
 {'id': 4,
  'name': 'Mark Andrews',
  'uid': 'mandrews',
  'position': 'Tight End',
  'team': 'Baltimore Ravens',
  'number': 89},
 {'id': 5,
  'name': 'Jonathan Taylor',
  'uid': 'jtaylor',
  'position': 'Running Back',
  'team': 'Indianapolis Colts',
  'number': 28},
 {'id': 6,
  'name': 'Daron Payne',
  'uid': 'dpayne',
  'position': 'Defensive Tackle',
  'team': 'Washington Commanders',
  'number': 94},
 {'id': 7,
  'name': 'Damar Hamlin',
  'uid': 'dhamlin',
  'position': 'Free Safety',
  'team': 'Buffalo Bills',
  'number': 3}]

## Updating users table in sqlite.db

In [12]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def update():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    if user != None:
        pass
    else:
        print(f"No user id {uid} found")
        return
    name = input("Enter your player's name:")
    position = input("Enter your player's position:")
    team = input("Enter your player's team:")
    number = input("Enter your player's jersey number:")
    
    # Initialize User object before date
    user = NFL(name=name, 
                uid=uid, 
                position=position,
                team=team,
                number=number,
                )
    # write object to database
    with app.app_context():
        try:
            object = user.update()
            print("Updated\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
update()

Updated
 {'id': None, 'name': 'Sauce Gardner', 'uid': 'sgardner', 'position': 'Cornerback', 'team': 'New York Jets', 'number': '1'}


## Deleting users table in sqlite.db

In [15]:
import sqlite3

database = 'instance/sqlite.db' # this is location of database

def delete():
    id = input("Enter id to delete")

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()
    
    try:
        cursor.execute("DELETE FROM NFL WHERE id = ?", (id))
        if cursor.rowcount == 0:
            # The id was not found in the table
            print(f"No id {id} was not found in the table")
        else:
            # The id was found in the table and the row was deleted
            print(f"The row with id {id} was successfully deleted")
        conn.commit()
    except sqlite3.Error as error:
        print("Error while executing the DELETE:", error)
    finally:
        conn.commit()
        conn.close()
        
delete()

The row with id 5 was successfully deleted


# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.